In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from run_model import *

In [2]:
df_genaudit = pd.read_csv('/work/frink/ramprasad.sa/hallucinations_annotations/scripts/data/genaudit_annotations.csv')
df_genaudit.head()[:2]

,Unnamed: 0.1,Unnamed: 0,id,benchmark_dataset_name,origin,source_format,summary_origin,error_spans,error_category,error_origin,faithful,model,source,summary,corrected_summary
0,0,0,Rachel Usher#REDDIT-83:flanul2-ul2,genaudit,REDDIT,document,model,nobody,NaN,model,0,flanul2,both me and my girlfriend participate in winte...,I waited outside the locker rooms for my girlf...,I waited outside the locker rooms for my girlf...
1,1,1,Rachel Usher#REDDIT-83:llama70b-ul2,genaudit,REDDIT,document,model,NaN,NaN,model,1,llama70b,both me and my girlfriend participate in winte...,A high school student was waiting for his girl...,A high school student was waiting for his girl...


In [6]:
import json

with open('/work/frink/ramprasad.sa/hallucinations_annotations/scripts/data/prompt_templates.json' , 'r') as fp:
    prompt_templates = json.load(fp)

In [5]:
prompt_summary_template = data['prompt_template']
data

{'prompt_template': '{instruction}\nDocument:{source}\nSummary:',
 'probe_prompt_template': '{instruction}\nDocument:{document}',
 'prompt_summary_template_llama3': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a summarization system<|eot_id<|start_header_id|>user<|end_header_id|>{instruction}\nDocument:{source}\nSummary:<|eot_id|><|start_header_id|>assistant<|end_header_id|>'}

In [31]:
prompt_summary_template_llama3 = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a summarization system<|eot_id|>\
<|start_header_id|>user<|end_header_id|>{instruction}\nDocument:{source}\nSummary:<|eot_id|>\
<|start_header_id|>assistant<|end_header_id|>"""


In [7]:
model_class = ModelPrompt(model_name = 'llama3_8b')

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
model_class.init_prompt_template(prompt_templates['prompt_summary_template_llama3'])

PROMPT <|begin_of_text|><|start_header_id|>system<|end_header_id|>You are a summarization system<|eot_id<|start_header_id|>user<|end_header_id|>{instruction}
Document:{source}
Summary:<|eot_id|><|start_header_id|>assistant<|end_header_id|>


In [9]:
instruction = "Generate a summary of the following document"

In [10]:
model_class.init_instruction(instruction)

In [11]:
df_docs = df_genaudit.drop_duplicates(subset=['source'], keep='last')
df_docs

,Unnamed: 0.1,Unnamed: 0,id,benchmark_dataset_name,origin,source_format,summary_origin,error_spans,error_category,error_origin,faithful,model,source,summary,corrected_summary
7,7,7,Rachel Usher#REDDIT-83:chatgpt-ul2,genaudit,REDDIT,document,model,NaN,NaN,model,1,chatgpt,both me and my girlfriend participate in winte...,The author and his girlfriend both participate...,The author and his girlfriend both participate...
15,15,15,Rachel Usher#REDDIT-54:chatgpt-ul2,genaudit,REDDIT,document,model,NaN,NaN,model,1,chatgpt,i thought my act test was today. it was yester...,The author of the document mistakenly believed...,The author of the document mistakenly believed...
23,23,23,Rachel Usher#ACIBENCH-D2N097:chatgpt-ul2,genaudit,ACIBENCH,dialogue,model,NaN,NaN,model,1,chatgpt,"[doctor] elizabeth , how are you ? [patient] i...",The document is a conversation between a docto...,The document is a conversation between a docto...
31,31,31,Cynthia Lamanda#XSUM-34017556:geminipro-ul2,genaudit,XSUM,document,model,harsh,NaN,model,0,geminipro,"Scotland's workforce is ""caught in a cruel tra...",According to the charity Citizen Advice Scotla...,According to the charity Citizen Advice Scotla...
39,39,39,Rachel Usher#XSUM-12889429:geminipro-ul2,genaudit,XSUM,document,model,NaN,NaN,model,1,geminipro,Mining giant Rio Tinto has lowered its minimum...,Mining giant Rio Tinto lowered its minimum acc...,Mining giant Rio Tinto lowered its minimum acc...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,670,670,Rachel Usher#ACIBENCH-D2N123:geminipro-ul2,genaudit,ACIBENCH,dialogue,model,in stools vomiting,NaN,model,0,geminipro,[doctor] so tyler is a 56 -year-old male who p...,"Tyler, a 56-year-old male, presented with diff...","Tyler, a 56-year-old male, presented with diff..."
678,678,678,Cynthia Lamanda#ACIBENCH-D2N104:chatgpt-ul2,genaudit,ACIBENCH,dialogue,model,Doctor Judy Gomez,NaN,model,0,chatgpt,"[doctor] judy gomez , mrn 869723 . date of bir...",The document is a conversation between Doctor ...,The document is a conversation between a docto...
685,685,685,Cynthia Lamanda#ACIBENCH-D2N105:chatgpt-ul2,genaudit,ACIBENCH,dialogue,model,NaN,NaN,model,1,chatgpt,[doctor] patient's name is diana scott . date ...,The document is a patient note between a docto...,The document is a patient note between a docto...
693,693,693,Rachel Usher#XSUM-35643091:geminipro-ul2,genaudit,XSUM,document,model,NaN,NaN,model,1,geminipro,At Harper Adams University they are fitting tr...,Harper Adams University is implanting tracking...,Harper Adams University is implanting tracking...


In [16]:
import math 
from tqdm import tqdm
import uuid 

model_name = 'llama3_8b'
added_summaries = []
seen_docids = []

for idx, row in tqdm(df_docs.iterrows(), total=df_docs.shape[0]):
    docid = row['id']
    init_dict = {k :v for k , v in row.items()}
    if docid not in seen_docids:
        seen_docids.append(docid)
        prompt_values = {
            'instruction': instruction,
            'source': row['source']
            }
        summary = model_class.generate(prompt_values)
        if model_name == 'llama3_8b':
            summary = summary.split('.assistant')[0]
        summary = summary.strip()
        init_dict['summary'] = summary
        init_dict['model'] = model_name
        init_dict['error_spans'] = float('nan')
        init_dict['error_category'] = float('nan')
        init_dict['faithful'] = float('nan')
        init_dict['corrected_summary'] = float('nan')
        init_dict['id'] = str(uuid.uuid4())
        added_summaries.append(init_dict)
        

  0%|                                                                                                                                                                                                                         | 0/91 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|██▎                                                                                                                                                                                                              | 1/91 [00:13<20:28, 13.65s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  2%|█

In [17]:
import uuid

In [13]:
df_summaries = pd.DataFrame(added_summaries)

In [15]:
df_summaries['summary'].values

array(['The writer and his girlfriend participate in winter sports at their high school. He has a late meet at 8pm and is required to dress up, while his girlfriend has a normal practice at 3:15pm. He waits outside the locker rooms for her to finish getting changed, but she takes longer than expected. When she finally emerges, he gives her a goodbye kiss and walks outside, only to realize that his bus has left without him. He frantically calls friends and family, but no one answers. With no other option, he is forced to walk 6 miles in freezing temperatures, wearing only a thin dress shirt and no gloves or hat',
       'The writer mistakenly thought their ACT test was on the current day, but it was actually the previous day. They spent 30 minutes walking around a high school trying to open every door, only to discover the test was actually scheduled for Saturday, June 13',
       "Dr. Elizabeth conducts an annual exam for a 66-year-old patient with a history of depression and hypertens

In [45]:
df_summaries.to_csv('/work/frink/ramprasad.sa/hallucinations_annotations/scripts/data/llama3_8b_summaries.csv')




In [47]:
df_summaries['summary'].values[2]

"[doctor] elizabeth , how are you ? [patient] i'm doing okay . how are you ? [doctor] doing okay . so i know the nurse told you about dax . i'd like to tell dax a little bit about you , okay ? [patient] okay . [doctor] all right . elizabeth is a 66-year-old female with a past medical history of significant for depression and hypertension who presents for her annual exam . so elizabeth , it's been a year since i've seen you . [patient] mm-hmm . [doctor] how are you doing ? [patient] i'm doing well . there's been so many events now that we've been able to get vaccinated , i've been able to see my grandkids again , go to more birthday parties , it's been fantastic . [doctor] yeah , i know . i've ... i mean , we've had some communication over telehealth and that type of thing , but it's not the same as being in-person , so i'm happy to see you today . now tell me a little bit about , you know , we have n't really got a chance to talk about your depression . how are you ... how are you doin